In [2]:
import os
import glob
import pandas as pd

In [3]:
# Define working directory
os.chdir("C:/Users/samya/VS Code Projects/Python/Python Projects/Spotify_hit_predictor/backend")

In [4]:
ds00 = pd.read_csv('Original_data/dataset-of-00s.csv')
ds10 = pd.read_csv('Original_data/dataset-of-10s.csv')
ds60 = pd.read_csv('Original_data/dataset-of-60s.csv')
ds70 = pd.read_csv('Original_data/dataset-of-70s.csv')
ds80 = pd.read_csv('Original_data/dataset-of-80s.csv')
ds90 = pd.read_csv('Original_data/dataset-of-90s.csv')

merge = [ds00, ds10, ds60, ds70, ds80, ds90]
df = pd.concat(merge)
df.reset_index(drop=True, inplace=True)

In [5]:
df2 = df.drop(['track', 'artist', 'uri',"key", "tempo", "speechiness", "chorus_hit"], axis=1)

In [6]:
#Validation of correlation between variables
import numpy as np 
df3=df2.corr()

In [7]:
# Reformat data
data = df2.values
X = data[:, 0:11]  
y = data[:, 11]

print(data.shape, X.shape, y.shape)

(41106, 12) (41106, 11) (41106,)


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

## Data Preprocessing

In [9]:
#Scale features (X) using MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler(feature_range=(0,1)).fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test) 

X_train_scaled

array([[0.78947368, 0.52283395, 0.76172236, ..., 0.05569473, 0.8       ,
        0.06508876],
       [0.54048583, 0.83094117, 0.77921392, ..., 0.03957652, 0.8       ,
        0.04142012],
       [0.5       , 0.66588373, 0.7455139 , ..., 0.03176775, 0.8       ,
        0.03550296],
       ...,
       [0.57692308, 0.36777999, 0.68303866, ..., 0.09203629, 0.8       ,
        0.09467456],
       [0.41902834, 0.16070793, 0.50819858, ..., 0.0441886 , 0.8       ,
        0.04733728],
       [0.29554656, 0.24873856, 0.67937808, ..., 0.0506431 , 0.6       ,
        0.06508876]])

One-hot encode the labels

In [10]:
#One-hot encode output labels (y)
from tensorflow.keras.utils import to_categorical
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

y_train_categorical

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)

## Creating and defining our Deep Learning Model Architecture

In [11]:
#Create a sequential model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential() 


number_inputs = 11  

#Create hidden layers
model.add(Dense(units=14,activation='relu', input_dim=number_inputs))
model.add(Dense(units=120,activation='relu'))
model.add(Dense(units=80,activation='relu'))

#Create output layer
number_classes = 2
model.add(Dense(units=number_classes, activation='softmax')) 

Number of Hidden Nodes 
https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw#:~:text=The%20number%20of%20hidden%20neurons,size%20of%20the%20input%20layer.

In [12]:
#Model Summary
model.summary() 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 14)                168       
                                                                 
 dense_1 (Dense)             (None, 120)               1800      
                                                                 
 dense_2 (Dense)             (None, 80)                9680      
                                                                 
 dense_3 (Dense)             (None, 2)                 162       
                                                                 
Total params: 11,810
Trainable params: 11,810
Non-trainable params: 0
_________________________________________________________________


In [13]:
#Compile the Model
import tensorflow as tf

opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

## Training the Model

In [14]:
#Training the Model
history = model.fit(X_train_scaled, y_train_categorical, epochs=500, batch_size=2000, shuffle=True, verbose=2)

Epoch 1/500
16/16 - 0s - loss: 0.6792 - accuracy: 0.5761 - 387ms/epoch - 24ms/step
Epoch 2/500
16/16 - 0s - loss: 0.6267 - accuracy: 0.6598 - 47ms/epoch - 3ms/step
Epoch 3/500
16/16 - 0s - loss: 0.5844 - accuracy: 0.6795 - 50ms/epoch - 3ms/step
Epoch 4/500
16/16 - 0s - loss: 0.5586 - accuracy: 0.7058 - 60ms/epoch - 4ms/step
Epoch 5/500
16/16 - 0s - loss: 0.5445 - accuracy: 0.7200 - 50ms/epoch - 3ms/step
Epoch 6/500
16/16 - 0s - loss: 0.5325 - accuracy: 0.7290 - 48ms/epoch - 3ms/step
Epoch 7/500
16/16 - 0s - loss: 0.5219 - accuracy: 0.7380 - 51ms/epoch - 3ms/step
Epoch 8/500
16/16 - 0s - loss: 0.5155 - accuracy: 0.7412 - 48ms/epoch - 3ms/step
Epoch 9/500
16/16 - 0s - loss: 0.5124 - accuracy: 0.7440 - 32ms/epoch - 2ms/step
Epoch 10/500
16/16 - 0s - loss: 0.5091 - accuracy: 0.7438 - 51ms/epoch - 3ms/step
Epoch 11/500
16/16 - 0s - loss: 0.5076 - accuracy: 0.7471 - 41ms/epoch - 3ms/step
Epoch 12/500
16/16 - 0s - loss: 0.5056 - accuracy: 0.7476 - 53ms/epoch - 3ms/step
Epoch 13/500
16/16 - 0s

Epoch 101/500
16/16 - 0s - loss: 0.4812 - accuracy: 0.7666 - 42ms/epoch - 3ms/step
Epoch 102/500
16/16 - 0s - loss: 0.4829 - accuracy: 0.7649 - 34ms/epoch - 2ms/step
Epoch 103/500
16/16 - 0s - loss: 0.4830 - accuracy: 0.7637 - 42ms/epoch - 3ms/step
Epoch 104/500
16/16 - 0s - loss: 0.4829 - accuracy: 0.7633 - 40ms/epoch - 3ms/step
Epoch 105/500
16/16 - 0s - loss: 0.4819 - accuracy: 0.7656 - 33ms/epoch - 2ms/step
Epoch 106/500
16/16 - 0s - loss: 0.4810 - accuracy: 0.7646 - 42ms/epoch - 3ms/step
Epoch 107/500
16/16 - 0s - loss: 0.4808 - accuracy: 0.7648 - 40ms/epoch - 2ms/step
Epoch 108/500
16/16 - 0s - loss: 0.4806 - accuracy: 0.7654 - 33ms/epoch - 2ms/step
Epoch 109/500
16/16 - 0s - loss: 0.4801 - accuracy: 0.7672 - 47ms/epoch - 3ms/step
Epoch 110/500
16/16 - 0s - loss: 0.4802 - accuracy: 0.7662 - 35ms/epoch - 2ms/step
Epoch 111/500
16/16 - 0s - loss: 0.4821 - accuracy: 0.7633 - 41ms/epoch - 3ms/step
Epoch 112/500
16/16 - 0s - loss: 0.4808 - accuracy: 0.7659 - 46ms/epoch - 3ms/step
Epoc

Epoch 200/500
16/16 - 0s - loss: 0.4702 - accuracy: 0.7721 - 49ms/epoch - 3ms/step
Epoch 201/500
16/16 - 0s - loss: 0.4697 - accuracy: 0.7740 - 33ms/epoch - 2ms/step
Epoch 202/500
16/16 - 0s - loss: 0.4711 - accuracy: 0.7726 - 43ms/epoch - 3ms/step
Epoch 203/500
16/16 - 0s - loss: 0.4696 - accuracy: 0.7728 - 42ms/epoch - 3ms/step
Epoch 204/500
16/16 - 0s - loss: 0.4701 - accuracy: 0.7725 - 42ms/epoch - 3ms/step
Epoch 205/500
16/16 - 0s - loss: 0.4700 - accuracy: 0.7736 - 42ms/epoch - 3ms/step
Epoch 206/500
16/16 - 0s - loss: 0.4715 - accuracy: 0.7712 - 41ms/epoch - 3ms/step
Epoch 207/500
16/16 - 0s - loss: 0.4707 - accuracy: 0.7734 - 41ms/epoch - 3ms/step
Epoch 208/500
16/16 - 0s - loss: 0.4691 - accuracy: 0.7734 - 33ms/epoch - 2ms/step
Epoch 209/500
16/16 - 0s - loss: 0.4686 - accuracy: 0.7749 - 41ms/epoch - 3ms/step
Epoch 210/500
16/16 - 0s - loss: 0.4684 - accuracy: 0.7740 - 41ms/epoch - 3ms/step
Epoch 211/500
16/16 - 0s - loss: 0.4687 - accuracy: 0.7744 - 42ms/epoch - 3ms/step
Epoc

Epoch 299/500
16/16 - 0s - loss: 0.4606 - accuracy: 0.7786 - 42ms/epoch - 3ms/step
Epoch 300/500
16/16 - 0s - loss: 0.4604 - accuracy: 0.7791 - 33ms/epoch - 2ms/step
Epoch 301/500
16/16 - 0s - loss: 0.4598 - accuracy: 0.7794 - 51ms/epoch - 3ms/step
Epoch 302/500
16/16 - 0s - loss: 0.4605 - accuracy: 0.7797 - 33ms/epoch - 2ms/step
Epoch 303/500
16/16 - 0s - loss: 0.4601 - accuracy: 0.7801 - 60ms/epoch - 4ms/step
Epoch 304/500
16/16 - 0s - loss: 0.4594 - accuracy: 0.7804 - 42ms/epoch - 3ms/step
Epoch 305/500
16/16 - 0s - loss: 0.4608 - accuracy: 0.7788 - 36ms/epoch - 2ms/step
Epoch 306/500
16/16 - 0s - loss: 0.4599 - accuracy: 0.7795 - 50ms/epoch - 3ms/step
Epoch 307/500
16/16 - 0s - loss: 0.4611 - accuracy: 0.7790 - 41ms/epoch - 3ms/step
Epoch 308/500
16/16 - 0s - loss: 0.4595 - accuracy: 0.7786 - 51ms/epoch - 3ms/step
Epoch 309/500
16/16 - 0s - loss: 0.4596 - accuracy: 0.7797 - 40ms/epoch - 2ms/step
Epoch 310/500
16/16 - 0s - loss: 0.4608 - accuracy: 0.7783 - 32ms/epoch - 2ms/step
Epoc

Epoch 398/500
16/16 - 0s - loss: 0.4543 - accuracy: 0.7811 - 41ms/epoch - 3ms/step
Epoch 399/500
16/16 - 0s - loss: 0.4543 - accuracy: 0.7824 - 51ms/epoch - 3ms/step
Epoch 400/500
16/16 - 0s - loss: 0.4541 - accuracy: 0.7812 - 42ms/epoch - 3ms/step
Epoch 401/500
16/16 - 0s - loss: 0.4541 - accuracy: 0.7830 - 49ms/epoch - 3ms/step
Epoch 402/500
16/16 - 0s - loss: 0.4546 - accuracy: 0.7821 - 50ms/epoch - 3ms/step
Epoch 403/500
16/16 - 0s - loss: 0.4545 - accuracy: 0.7829 - 43ms/epoch - 3ms/step
Epoch 404/500
16/16 - 0s - loss: 0.4547 - accuracy: 0.7829 - 49ms/epoch - 3ms/step
Epoch 405/500
16/16 - 0s - loss: 0.4534 - accuracy: 0.7830 - 55ms/epoch - 3ms/step
Epoch 406/500
16/16 - 0s - loss: 0.4556 - accuracy: 0.7812 - 58ms/epoch - 4ms/step
Epoch 407/500
16/16 - 0s - loss: 0.4535 - accuracy: 0.7834 - 49ms/epoch - 3ms/step
Epoch 408/500
16/16 - 0s - loss: 0.4552 - accuracy: 0.7814 - 47ms/epoch - 3ms/step
Epoch 409/500
16/16 - 0s - loss: 0.4542 - accuracy: 0.7817 - 43ms/epoch - 3ms/step
Epoc

Epoch 497/500
16/16 - 0s - loss: 0.4494 - accuracy: 0.7843 - 51ms/epoch - 3ms/step
Epoch 498/500
16/16 - 0s - loss: 0.4485 - accuracy: 0.7843 - 41ms/epoch - 3ms/step
Epoch 499/500
16/16 - 0s - loss: 0.4482 - accuracy: 0.7856 - 50ms/epoch - 3ms/step
Epoch 500/500
16/16 - 0s - loss: 0.4489 - accuracy: 0.7846 - 50ms/epoch - 3ms/step


## Validation of the Model

In [15]:
#Evaluate the Model using the testing data
#Compare Model performace between training and testing data
model_loss_train, model_accuracy_train = model.evaluate(X_train_scaled, y_train_categorical, verbose=2)
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)

print(f"Train --> Loss: {model_loss_train}, Accuracy: {model_accuracy_train}")    
print(f"Test --> Loss: {model_loss}, Accuracy: {model_accuracy}")

964/964 - 1s - loss: 0.4490 - accuracy: 0.7848 - 876ms/epoch - 909us/step
322/322 - 0s - loss: 0.4743 - accuracy: 0.7740 - 259ms/epoch - 805us/step
Train --> Loss: 0.4489516615867615, Accuracy: 0.7847805619239807
Test --> Loss: 0.47426852583885193, Accuracy: 0.7739612460136414


## Saving the Trained Model

In [16]:
# Save the model
model.save("../Models/h5/skey&tempo&speech&chorus.h5")